In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os

import click
import pandas as pd
from tqdm import tqdm
import numpy as np
import json

from ccd.parking_zone_evaluator import ParkingZoneEvaluator
from ccd.utils import read_layers_from_gdb
from ccd.constants import COLUMNS
from ccd.rule import Rule
from ccd.constants import REGULATIONS

from curblr import Regulation
from curblr.rule import Rule as CLRRule

In [ ]:
geodatabase = '../data/raw/ParkingZones.gdb/'
layers = read_layers_from_gdb(geodatabase)

In [ ]:
pz = layers['Parking_Zones']
pzr = layers['Parking_Zone_Regulations']
pza = layers['Parking_Zones__ATTACH']

pze = ParkingZoneEvaluator(pz, pzr, pza)

In [ ]:
all_dfs = []
all_ids = pz['GlobalID'].values
fname = 'output-all.csv'

In [ ]:
all_ids = all_ids[0:50]
fname = fname.replace('all', 'test')

#### PT 1

In [ ]:
cats = {'parking':[], 'no parking': [], 'standing': [], 'no standing': [], 'loading': [], 'no loading': []}
unclassified = []

regs = []
id_to_reg = {}
for _, r in pzr.iterrows():
    reg = r['Regulation'].lower().strip()
    if reg == 'other (see notes)':
        note = r['Notes']
        if note:
            regs.append(note)
        else:
            regs.append('no regulations listed')
    else:
        regs.append(reg)
regs = list(set(regs))

for reg in regs:
    rc = None
    if reg:
        rc = reg.lower()
    
    if 'loading' in rc and not 'no loading' in rc:
        cats['loading'].append(reg)
        continue
        
    if 'no loading' in rc:
        cats['no loading'].append(reg)
        continue
        
    if rc == 'stopping' or rc == 'standing':
        cats['standing'].append(reg)
        continue
    
    if rc == 'no stopping (except bicycles)':
        cats['standing'].append(reg)
        continue
    
    if 'no stopping' in rc or 'no standing' in rc:
        cats['no standing'].append(reg)
        continue
        
    if 'no parking' in rc or 'parking prohibited' in rc:
        cats['no parking'].append(reg)
        continue
    
    if rc == 'bicycles only':
        cats['parking'].append(reg)
        continue
    
    if 'only' in rc:
        cats['parking'].append(reg)
        continue
    
    if rc in ['bus zone', 'indego bike zone', 'fire zone', 'snow emergency zone', 'ambulance zone']:
        cats['no parking'].append(reg)
        continue
    
    if 'not valid' in rc:
        cats['no parking'].append(reg)
        continue
        
    if 'drop off' in rc or 'pickup' in rc:
        cats['standing'].append(reg)
        continue
    
    if 'prohibited' in rc:
        cats['no parking'].append(reg)
        continue
        
    if rc == 'no regulations listed':
        cats['parking'].append(reg)
        continue
    
    if rc in ('no bus parking', 'no truck parking'):
        cats['no parking'].append(reg)
        continue
    
    if 'parking' in rc:
        cats['parking'].append(reg)
        continue
    
    if rc == 'no truck idling':
        cats['no standing'].append(reg)
        continue
    
    unclassified.append(reg)

In [ ]:
d = {}
for activity, rules in cats.items():
    for rule in rules:
        d[rule] = {'activity': activity}
        rc = rule.lower()
        if rc == activity:
            d[rule]['reason'] = None
        else:
            d[rule]['reason'] = rc

In [ ]:
classes = pd.read_csv('../data/interim/regulation_to_userclass.csv')

In [ ]:
for _, r in classes.iterrows():
    reg = r['reg_c']
    c = r['class']
    if not isinstance(c, str):
        if np.isnan(c):
            c = None
    d[reg]['classes'] = [c]

In [ ]:
with open('../data/interim/reg_lookup.json', 'w') as f:
    json.dump(d, f)

#### PT 2

In [ ]:
from curblr.timespan import DaysOfWeek

In [ ]:
with open('../data/interim/reg_lookup.json') as f:
    d = json.load(f)

In [ ]:
r = Rule(pzr.iloc[2])

In [ ]:
r.ptr_secondary[0].all_time

In [ ]:
days_of_week = DaysOfWeek(r.ptr_primary[0].days)